In [35]:
import pandas as pd
import numpy as np


def filter_dataframes(dataframes, query_string):
    """
    Filter a list of DataFrames using a query string.

    Parameters:
        dataframes (dict): Dictionary of pandas DataFrames.
        query_string (str): Query string to filter the DataFrames.

    Returns:
        list: List of filtered DataFrames.
    """
    filtered_dataframes = {key: df.query(query_string) for key, df in dataframes.items()}
    return filtered_dataframes


def calculate_sum_columns(dataframes_dict, columns):
    """
    Calculate the sum of specified columns for each DataFrame in a dictionary.

    Parameters:
        dataframes_dict (dict): Dictionary of pandas DataFrames.
        columns (list): List of column names for which sum needs to be calculated.

    Returns:
        pandas DataFrame: DataFrame containing the sums of specified columns for each DataFrame,
                          with DataFrame keys as index.
    """
    # Dictionary to store sums for each DataFrame
    sums = {}

    # Calculate sum for each DataFrame
    for key, df in dataframes_dict.items():
        # Select specified columns and calculate sum
        sums[key] = df[columns].sum()

    # Concatenate sums into a DataFrame
    result_df = pd.concat(sums, axis=1).T

    return result_df


ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [36]:
COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [37]:
df = pd.read_excel(
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
).iloc[:-2]

df = df.rename(columns={'Sexo\n1=Hombre\n2=Mujer': "hombre_mujer"})
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

# Cálculo de Personas por Estrato

Aquí se quiere calcular la cantidad de personas que hay en los siguientes estratos:

1. Hombres y Mujeres
2. Hombres
3. Mujeres
4. Población sobre 45 años
5. Población sobre 60 años
6. Población entre 60 a 70 años
7. Población recién nacidos
8. Población sobre 65 años

Para cada estrato se calculará la población que hay para el SSMO, Región Metropolitana y el País.

In [38]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

In [39]:
result_df = calculate_sum_columns(ESTRATOS_A_CALCULAR, COLUMNAS_POBLACION)

## 1. Hombres y Mujeres

In [46]:
query_string = "hombre_mujer == 1"
dfs_hombres = filter_dataframes(ESTRATOS_A_CALCULAR, query_string)
suma_hombres = calculate_sum_columns(dfs_hombres, COLUMNAS_POBLACION)

In [47]:
suma_hombres

,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021,Poblacion 2022,Poblacion 2023,Poblacion 2024,Poblacion 2025,Poblacion 2026,Poblacion 2027,Poblacion 2028,Poblacion 2029,Poblacion 2030,Poblacion 2031,Poblacion 2032,Poblacion 2033,Poblacion 2034,Poblacion 2035
Pais,9074217,9244484,9424139,9599101,9708512,9782978,9848466,9910500,9970025,10027134,10081754,10133749,10183028,10229496,10273948,10315745,10354845,10391234,10424918
Tarapacá,173093,179390,186270,193145,197277,199973,202347,204628,206868,209068,211228,213341,215413,217437,219443,221420,223367,225273,227146
Antofagasta,315354,325309,336667,348210,354057,356913,359072,361009,362842,364581,366228,367778,369235,370605,372109,373359,374525,375592,376567
Atacama,154008,155595,157283,158939,159897,160468,160922,161331,161704,162029,162315,162567,162783,162961,163123,163248,163348,163413,163451
Coquimbo,388759,395636,402663,409614,415361,420466,425400,430248,435021,439728,444371,448935,453433,457866,462277,466601,470839,474969,479005
Valparaíso,921681,934075,946578,958672,968008,975818,983195,990348,997310,1004078,1010648,1016996,1023125,1029027,1034769,1040293,1045594,1050666,1055500
Libertador General Bernardo O'Higgins,477231,483455,489662,495674,500528,504711,508672,512497,516204,519785,523235,526553,529715,532724,535625,538396,541031,543523,545888
Maule,537574,543871,550096,556100,561122,565612,569896,574048,578091,581994,585758,589356,592806,596108,599313,602374,605294,608067,610686
Biobío,797987,802348,806588,810577,813546,815898,818002,819922,821640,823158,824452,825521,826362,826973,827396,827587,827550,827275,826756
La Araucanía,486792,489833,492834,495735,498048,499978,501761,503438,505021,506500,507873,509138,510266,511270,512173,512959,513614,514155,514577


## 2. Hombres

In [ ]:
estratos_hombres = map(lambda x: x[x["Sexo\n1=Hombre\n2=Mujer"] == 1.0], estratos_a_calcular)
suma_hombres = map(calcular_suma_poblacion_anios, estratos_hombres)
imprimir_resultados_suma_estratos(suma_hombres, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,660253
1,Poblacion 2018,677954
2,Poblacion 2019,697445
3,Poblacion 2020,716354
4,Poblacion 2021,726815
5,Poblacion 2022,733589
6,Poblacion 2023,739367
7,Poblacion 2024,744846
8,Poblacion 2025,750142
9,Poblacion 2026,755304


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,3690365
1,Poblacion 2018,3793180
2,Poblacion 2019,3903339
3,Poblacion 2020,4010416
4,Poblacion 2021,4071264
5,Poblacion 2022,4107724
6,Poblacion 2023,4138275
7,Poblacion 2024,4166844
8,Poblacion 2025,4194152
9,Poblacion 2026,4220294


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,9074217
1,Poblacion 2018,9244484
2,Poblacion 2019,9424139
3,Poblacion 2020,9599101
4,Poblacion 2021,9708512
5,Poblacion 2022,9782978
6,Poblacion 2023,9848466
7,Poblacion 2024,9910500
8,Poblacion 2025,9970025
9,Poblacion 2026,10027134


## 3. Mujeres

In [ ]:
estratos_mujeres = map(lambda x: x[x["Sexo\n1=Hombre\n2=Mujer"] == 2.0], estratos_a_calcular)
suma_mujeres = map(calcular_suma_poblacion_anios, estratos_mujeres)
# imprimir_resultados_suma_estratos(suma_mujeres, nombres_estratos)

## 4. Población sobre 45 años

In [ ]:
estratos_mayores_45 = map(lambda x: x[x["Edad"] > 45], estratos_a_calcular)
suma_mayores_45 = map(calcular_suma_poblacion_anios, estratos_mayores_45)
# imprimir_resultados_suma_estratos(suma_mayores_45, nombres_estratos)

## 5. Población sobre 60 años

In [ ]:
estratos_mayores_60 = map(lambda x: x[x["Edad"] > 60], estratos_a_calcular)
suma_mayores_60 = map(calcular_suma_poblacion_anios, estratos_mayores_60)
# imprimir_resultados_suma_estratos(suma_mayores_60, nombres_estratos)

## 6. Población entre 60 a 70 años

In [ ]:
estratos_entre_60_a_70 = map(
    lambda x: x[(x["Edad"] >= 60) & (x["Edad"] <= 70)], estratos_a_calcular
)
suma_entre_60_a_70 = map(calcular_suma_poblacion_anios, estratos_entre_60_a_70)
# imprimir_resultados_suma_estratos(suma_entre_60_a_70, nombres_estratos)

## 7. Población Recién Nacidos Vivos

In [ ]:
estratos_recien_nacidos = map(lambda x: x[x["Edad"] == 0], estratos_a_calcular)
suma_recien_nacidos = map(calcular_suma_poblacion_anios, estratos_recien_nacidos)
imprimir_resultados_suma_estratos(suma_recien_nacidos, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,16070
1,Poblacion 2018,16006
2,Poblacion 2019,16346
3,Poblacion 2020,16916
4,Poblacion 2021,17242
5,Poblacion 2022,17290
6,Poblacion 2023,17206
7,Poblacion 2024,17070
8,Poblacion 2025,16903
9,Poblacion 2026,16700


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,97619
1,Poblacion 2018,97355
2,Poblacion 2019,99527
3,Poblacion 2020,103124
4,Poblacion 2021,105341
5,Poblacion 2022,105844
6,Poblacion 2023,105562
7,Poblacion 2024,104957
8,Poblacion 2025,104133
9,Poblacion 2026,103110


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,232425
1,Poblacion 2018,227660
2,Poblacion 2019,229927
3,Poblacion 2020,234728
4,Poblacion 2021,237485
5,Poblacion 2022,237819
6,Poblacion 2023,237015
7,Poblacion 2024,235715
8,Poblacion 2025,234060
9,Poblacion 2026,232106


## 8. Población sobre 65 años

In [ ]:
estratos_mayores_a_65 = map(lambda x: x[x["Edad"] > 65], estratos_a_calcular)
suma_mayores_a_65 = map(calcular_suma_poblacion_anios, estratos_mayores_a_65)
# imprimir_resultados_suma_estratos(suma_mayores_a_65, nombres_estratos)

## 9. Población entre 15 a 24 años

In [ ]:
estratos_entre_15_a_24 = map(
    lambda x: x[(x["Edad"] >= 15) & (x["Edad"] <= 24)], estratos_a_calcular
)
suma_entre_15_a_24 = map(calcular_suma_poblacion_anios, estratos_entre_15_a_24)
imprimir_resultados_suma_estratos(suma_entre_15_a_24, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,189742
1,Poblacion 2018,190954
2,Poblacion 2019,191375
3,Poblacion 2020,190824
4,Poblacion 2021,186919
5,Poblacion 2022,182373
6,Poblacion 2023,178765
7,Poblacion 2024,176246
8,Poblacion 2025,174840
9,Poblacion 2026,173985


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,1130890
1,Poblacion 2018,1138954
2,Poblacion 2019,1142179
3,Poblacion 2020,1140074
4,Poblacion 2021,1118872
5,Poblacion 2022,1093746
6,Poblacion 2023,1074338
7,Poblacion 2024,1061346
8,Poblacion 2025,1055025
9,Poblacion 2026,1051726


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,2735974
1,Poblacion 2018,2730638
2,Poblacion 2019,2719952
3,Poblacion 2020,2703243
4,Poblacion 2021,2658934
5,Poblacion 2022,2610833
6,Poblacion 2023,2575212
7,Poblacion 2024,2553410
8,Poblacion 2025,2542575
9,Poblacion 2026,2537479


## 10. Población entre 25 a 44 años

In [ ]:
estratos_entre_25_a_44 = map(
    lambda x: x[(x["Edad"] >= 25) & (x["Edad"] <= 44)], estratos_a_calcular
)
suma_entre_25_a_44 = map(calcular_suma_poblacion_anios, estratos_entre_25_a_44)
imprimir_resultados_suma_estratos(suma_entre_25_a_44, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,436427
1,Poblacion 2018,453426
2,Poblacion 2019,473542
3,Poblacion 2020,492775
4,Poblacion 2021,501932
5,Poblacion 2022,505548
6,Poblacion 2023,507208
7,Poblacion 2024,507847
8,Poblacion 2025,507166
9,Poblacion 2026,505227


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,2385658
1,Poblacion 2018,2485423
2,Poblacion 2019,2600727
3,Poblacion 2020,2711382
4,Poblacion 2021,2767080
5,Poblacion 2022,2790584
6,Poblacion 2023,2803082
7,Poblacion 2024,2809600
8,Poblacion 2025,2808483
9,Poblacion 2026,2799916


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,5547955
1,Poblacion 2018,5700544
2,Poblacion 2019,5868456
3,Poblacion 2020,6027605
4,Poblacion 2021,6112626
5,Poblacion 2022,6154579
6,Poblacion 2023,6180898
7,Poblacion 2024,6196537
8,Poblacion 2025,6198781
9,Poblacion 2026,6187809


## 11. Población entre 45 a 59 años

In [ ]:
estratos_entre_45_a_59 = map(
    lambda x: x[(x["Edad"] >= 45) & (x["Edad"] <= 59)], estratos_a_calcular
)
suma_entre_45_a_59 = map(calcular_suma_poblacion_anios, estratos_entre_45_a_59)
imprimir_resultados_suma_estratos(suma_entre_45_a_59, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,245976
1,Poblacion 2018,250759
2,Poblacion 2019,256118
3,Poblacion 2020,261703
4,Poblacion 2021,265636
5,Poblacion 2022,268701
6,Poblacion 2023,271280
7,Poblacion 2024,273470
8,Poblacion 2025,275767
9,Poblacion 2026,278503


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,1377029
1,Poblacion 2018,1403720
2,Poblacion 2019,1433262
3,Poblacion 2020,1465104
4,Poblacion 2021,1489209
5,Poblacion 2022,1508677
6,Poblacion 2023,1525723
7,Poblacion 2024,1540794
8,Poblacion 2025,1556504
9,Poblacion 2026,1574671


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,3464640
1,Poblacion 2018,3520370
2,Poblacion 2019,3580096
3,Poblacion 2020,3641414
4,Poblacion 2021,3688895
5,Poblacion 2022,3725231
6,Poblacion 2023,3754587
7,Poblacion 2024,3780743
8,Poblacion 2025,3807273
9,Poblacion 2026,3837634


## 12. Población entre 60 a 74 años

In [ ]:
estratos_entre_60_a_74 = map(
    lambda x: x[(x["Edad"] >= 60) & (x["Edad"] <= 74)], estratos_a_calcular
)
suma_entre_60_a_74 = map(calcular_suma_poblacion_anios, estratos_entre_60_a_74)
imprimir_resultados_suma_estratos(suma_entre_60_a_74, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,167310
1,Poblacion 2018,174825
2,Poblacion 2019,181955
3,Poblacion 2020,189002
4,Poblacion 2021,195558
5,Poblacion 2022,201828
6,Poblacion 2023,207959
7,Poblacion 2024,214244
8,Poblacion 2025,220109
9,Poblacion 2026,225053


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,830100
1,Poblacion 2018,866475
2,Poblacion 2019,900903
3,Poblacion 2020,935403
4,Poblacion 2021,968289
5,Poblacion 2022,999718
6,Poblacion 2023,1030642
7,Poblacion 2024,1062262
8,Poblacion 2025,1091748
9,Poblacion 2026,1117003


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,2121233
1,Poblacion 2018,2212101
2,Poblacion 2019,2299723
3,Poblacion 2020,2387391
4,Poblacion 2021,2473219
5,Poblacion 2022,2557907
6,Poblacion 2023,2642414
7,Poblacion 2024,2727429
8,Poblacion 2025,2807932
9,Poblacion 2026,2879618


## 13. Población mayor o igual a 75 años

In [ ]:
estratos_mayores_a_75 = map(lambda x: x[x["Edad"] >= 75], estratos_a_calcular)
suma_mayores_a_75 = map(calcular_suma_poblacion_anios, estratos_mayores_a_75)
imprimir_resultados_suma_estratos(suma_mayores_a_75, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,84276
1,Poblacion 2018,87041
2,Poblacion 2019,89851
3,Poblacion 2020,92865
4,Poblacion 2021,96127
5,Poblacion 2022,99705
6,Poblacion 2023,103512
7,Poblacion 2024,107307
8,Poblacion 2025,111601
9,Poblacion 2026,116826


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,326982
1,Poblacion 2018,337737
2,Poblacion 2019,348809
3,Poblacion 2020,360939
4,Poblacion 2021,374142
5,Poblacion 2022,388666
6,Poblacion 2023,404171
7,Poblacion 2024,419477
8,Poblacion 2025,436847
9,Poblacion 2026,458080


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,859688
1,Poblacion 2018,891612
2,Poblacion 2019,924817
3,Poblacion 2020,960619
4,Poblacion 2021,999024
5,Poblacion 2022,1040647
6,Poblacion 2023,1084911
7,Poblacion 2024,1130233
8,Poblacion 2025,1180605
9,Poblacion 2026,1239443


## Poblacion sobre desde 15 anios o mas

In [ ]:
estratos_mayores_a_15 = map(lambda x: x[x["Edad"] >= 15], estratos_a_calcular)
suma_mayores_a_15 = map(calcular_suma_poblacion_anios, estratos_mayores_a_15)
imprimir_resultados_suma_estratos(suma_mayores_a_15, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,1123731
1,Poblacion 2018,1157005
2,Poblacion 2019,1192841
3,Poblacion 2020,1227169
4,Poblacion 2021,1246172
5,Poblacion 2022,1258155
6,Poblacion 2023,1268724
7,Poblacion 2024,1279114
8,Poblacion 2025,1289483
9,Poblacion 2026,1299594


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,6050659
1,Poblacion 2018,6232309
2,Poblacion 2019,6425880
3,Poblacion 2020,6612902
4,Poblacion 2021,6717592
5,Poblacion 2022,6781391
6,Poblacion 2023,6837956
7,Poblacion 2024,6893479
8,Poblacion 2025,6948607
9,Poblacion 2026,7001396


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,14729490
1,Poblacion 2018,15055265
2,Poblacion 2019,15393044
3,Poblacion 2020,15720272
4,Poblacion 2021,15932698
5,Poblacion 2022,16089197
6,Poblacion 2023,16238022
7,Poblacion 2024,16388352
8,Poblacion 2025,16537166
9,Poblacion 2026,16681983


In [ ]:
estratos_mayores_a_20 = map(lambda x: x[x["Edad"] >= 20], estratos_a_calcular)
suma_mayores_a_20 = map(calcular_suma_poblacion_anios, estratos_mayores_a_20)
imprimir_resultados_suma_estratos(suma_mayores_a_20, nombres_estratos)

La suma de poblaciones para el estrato SSMO es:


,index,0
0,Poblacion 2017,1038531
1,Poblacion 2018,1071960
2,Poblacion 2019,1108254
3,Poblacion 2020,1143080
4,Poblacion 2021,1163186
5,Poblacion 2022,1175921
6,Poblacion 2023,1186599
7,Poblacion 2024,1196385
8,Poblacion 2025,1205562
9,Poblacion 2026,1214636


La suma de poblaciones para el estrato Region Metropolitana es:


,index,0
0,Poblacion 2017,5540099
1,Poblacion 2018,5722755
2,Poblacion 2019,5919183
3,Poblacion 2020,6109043
4,Poblacion 2021,6219576
5,Poblacion 2022,6286901
6,Poblacion 2023,6342967
7,Poblacion 2024,6393780
8,Poblacion 2025,6440664
9,Poblacion 2026,6486377


La suma de poblaciones para el estrato Pais es:


,index,0
0,Poblacion 2017,13432665
1,Poblacion 2018,13768853
2,Poblacion 2019,14120077
3,Poblacion 2020,14460210
4,Poblacion 2021,14689893
5,Poblacion 2022,14857110
6,Poblacion 2023,15002663
7,Poblacion 2024,15138270
8,Poblacion 2025,15266800
9,Poblacion 2026,15392634


In [ ]:
grouped_df = (
    df_ssmo.groupby(["Nombre Comuna", "grupo_etario_poblacion"])[COLUMNAS_POBLACION]
    .sum()
    .reset_index()
)
total_sum_by_comuna = df_ssmo.groupby(["Nombre Comuna"])[COLUMNAS_POBLACION].sum()
total_sum_by_comuna.insert(0, "grupo_etario_poblacion", "Total")
total_sum_by_comuna = total_sum_by_comuna.reset_index()

# Une grupos y totales de subgrupos
union_de_grupos = pd.concat([grouped_df, total_sum_by_comuna])
union_de_grupos["grupo_etario_poblacion"] = pd.Categorical(
    union_de_grupos["grupo_etario_poblacion"], ["Infantil", "Adulto", "Total"]
)
union_de_grupos = union_de_grupos.sort_values(["Nombre Comuna", "grupo_etario_poblacion"])

In [ ]:
union_de_grupos.to_excel("../data/interim/resumen_poblacion_INE_SSMO.xlsx", index=False)

In [ ]:
# Obtiene el resumen por RM
sum_adulto_infantil_rm = df_rm.groupby("grupo_etario_poblacion")[COLUMNAS_POBLACION].sum()
total_sum_rm = sum_adulto_infantil_rm.sum()
sum_adulto_infantil_rm.loc[len(sum_adulto_infantil_rm)] = total_sum_rm

In [ ]:
sum_adulto_infantil_rm.to_excel("../data/interim/resumen_poblacion_INE_RM.xlsx")

In [ ]:
# Obtiene el resumen por Pais
sum_adulto_infantil_pais = df_pais.groupby("grupo_etario_poblacion")[COLUMNAS_POBLACION].sum()
total_sum_pais = sum_adulto_infantil_pais.sum()
sum_adulto_infantil_pais.loc[len(sum_adulto_infantil_pais)] = total_sum_pais

In [ ]:
sum_adulto_infantil_pais.to_excel("../data/interim/resumen_poblacion_INE_pais.xlsx")

In [ ]:
len(df["M"].value_counts())

16